In [ ]:
import sys, json

sys.path.insert(0, "../")

from lib import *
np.seterr(divide='ignore', invalid='ignore')

In [ ]:
user_input = {
    "config_file": "hd_1x2x6/hd_1x2x6_config",
    "root_file": ["Marley"],
    "rewrite": True,
    "debug": True,
}
info = json.load(open("../config/" + user_input["config_file"] + ".json", "r"))
data_filter = {
    "max_energy": 20,
    "min_energy": 0,
    "pre_nhits": 3,
    "primary": True,
    "neutron": False,
}
true, data, filter_idx = compute_root_workflow(
    user_input, info, data_filter, workflow="SMEARING", debug=user_input["debug"]
)
rprint(
    "-> Found %i electron candidates out of %i events!"
    % (len(filter_idx), data["Event"].size)
)

In [ ]:
list_hist = []
output_dict = {}

variable = "TotalEnergy"

config = user_input["config_file"].split("/")[0]
fit = {"color":"grey","threshold":0.3,"trimm":10, "spec_type":"max", "print":False}

analysis_info = json.load(open("../import/analysis.json", "r"))
energy_edges = np.linspace(
    analysis_info["RECO_ENERGY_RANGE"][0],
    analysis_info["RECO_ENERGY_RANGE"][1],
    analysis_info["RECO_ENERGY_BINS"]+ 1,
)
energy_centers = (energy_edges[1:] + energy_edges[:-1]) / 2
bin_width = energy_edges[1] - energy_edges[0]

eff_flux = get_detected_solar_spectrum(bins=energy_centers, components=["b8", "hep"])
eff_flux_b8 = get_detected_solar_spectrum(bins=energy_centers, components=["b8"])
eff_flux_hep = get_detected_solar_spectrum(bins=energy_centers, components=["hep"])

for idx, energy_bin in enumerate(energy_centers):
    neutrino_energy_filter = (data["TNuE"] > (energy_bin - bin_width / 2)) * (
        data["TNuE"] < (energy_bin + bin_width / 2)
    )  # Filtering genereted neutrinos in 1GeV energy bin
    this_filter = neutrino_energy_filter
    hist, bin_edges = np.histogram(
        # data[info["DEFAULT_ANALYSIS_ENERGY"]][this_filter], bins=energy_edges
        data[variable][this_filter], bins=energy_edges
    )

    hist = hist / np.sum(hist)
    hist = np.nan_to_num(hist, 0)
    flux = hist * eff_flux[idx]
    fluxb8 = hist * eff_flux_b8[idx]
    fluxhep = hist * eff_flux_hep[idx]

    list_hist.append(
        {
            "Geometry": info["GEOMETRY"],
            "Version": info["VERSION"],
            "Name": user_input["root_file"][0],
            "Generator": 1,
            "TrueEnergy": energy_bin,
            "Hist": hist,
            "Flux": flux,
            "FluxB8": fluxb8,
            "FluxHEP": fluxhep,
            "RecoEnergy": energy_centers,
        }
    )
    fit["func"] = "gauss"
    fig = make_subplots(rows=1, cols=1, subplot_titles=("Charge Calibration"))
    fig, popt, perr = get_hist1d_fit(
        # data[info["DEFAULT_ANALYSIS_ENERGY"]][this_filter],
        data[variable][this_filter],
        fig=fig,
        idx=(1, 1),
        per=[1, 99],
        acc=energy_edges,
        fit=fit,
        debug=False,
    )
    if len(popt) == 0:
        popt = [0, 0, 0]
        perr = [0, 0, 0]
    
    fig.update_layout(
        coloraxis=dict(colorscale="Turbo", colorbar=dict(title="Counts")),
        showlegend=False,
        title="Calibration",
        xaxis1_title="Reco Electron Energy [MeV]",
        yaxis1_title="Norm.",
    )

    fig = format_coustom_plotly(fig)
    if not os.path.exists("../images/calibration/%s_calibration/%s_calibration_mono/"%(config,config)):
        os.makedirs("../images/calibration/%s_calibration/%s_calibration_mono/"%(config,config))
    fig.write_image("../images/calibration/%s_calibration/%s_calibration_mono/%s_calibration_%.2f.png"%(config,config,config,energy_bin), width=2400, height=1080)
    # print("-> Saved images to ../images/calibration/%s_calibration/%s_calibration_mono/%s_calibration_%.2f.png"%(config,config,config,energy_bin))
    output_dict["{:.2f}".format(energy_bin)] = {"ENERGY":energy_bin,"SIGMA":popt[2],"SIGMA_ERR":perr[2]}

with open("../config/"+config+"/"+config+"_calib/"+config+"_mono_energy_resolution.json",'w') as f: json.dump(output_dict, f)

In [ ]:
df = pd.DataFrame(list_hist)
norm_smearing_df = df.copy()

smearing_df = pd.DataFrame(list(df["Flux"]),columns=energy_centers,index=energy_centers).T
norm_smearing_df = pd.DataFrame(list(norm_smearing_df["Hist"]),columns=energy_centers,index=energy_centers).T
smearing_df.to_pickle("../config/"+config+"/"+config+"_calib/"+config+"_smearing.pkl")
norm_smearing_df.to_pickle("../config/"+config+"/"+config+"_calib/"+config+"_norm_smearing.pkl")

fig = px.imshow(smearing_df,
    origin="lower",
    aspect=None,
    # facet_col="Name",
    color_continuous_scale="turbo",
    labels=dict(y="Reco Energy [MeV]",x="True Energy [MeV]"))


fig = format_coustom_plotly(fig,title="Smearing Matrix %s"%(config))
# fig.update_layout(coloraxis=dict(colorscale="Turbo",colorbar=dict(title="Flux Hz/MeV")),showlegend=False,)
# fig.write_image("../images/calibration/%s_calibration/%s_smearing.png"%(config,config))
# print_colored("-> Saved images to ../images/calibration/%s_calibration/%s_smearing.png"%(config,config),"SUCCESS")
fig.show()

In [ ]:
this_df = df.explode(["Flux","Hist","RecoEnergy"])
this_df["TrueEnergy"] = this_df["TrueEnergy"].astype(float)
print(this_df.groupby("Version")["Flux"].sum())
fig=px.bar(this_df,
    x="RecoEnergy",
    y="Flux",
    log_y=False,
    color="TrueEnergy",
    barmode="stack",
    template="presentation",
    facet_col="Version",
    color_continuous_scale="turbo"
    )

# 'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid', 'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr','ylorrd'
fig = format_coustom_plotly(fig,log=(False,False),tickformat=(None, ".1s"),matches=(None,None))
fig.update_layout(bargap=0)
fig.update_layout(coloraxis=dict(colorscale="Turbo",colorbar=dict(title="Energy")),showlegend=False,
    title="Reconstruction Smearing",
    xaxis1_title="Reco Electron Energy [MeV]",
    yaxis1_title="Norm.",
    )
if user_input["rewrite"]:
    fig.write_image("../images/calibration/%s_calibration/%s_smearing_hist.png"%(config,config))
    rprint("-> Saved images to ../images/calibration/%s_calibration/%s_smearing_hist.png"%(config,config))
fig.show()